In [11]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

def phases(iq: np.ndarray,start:int):
    list = []
    for i in range(start,len(iq)-2,6):
        list.append(
            np.angle(complex(iq[i],iq[i+1]))
        )
    return np.array(list)



antenna_conf= "2-2-0-5-6"

data = pd.read_json("../folder/data.json")
data = data[data["Switch Order"] == antenna_conf]
#data = data[data["TAG"] != "B"]
#data = data[data["rssi"] >-70]
del data["level_0"]
del data["index"]
data=data.reset_index()


# On IQ samples

In [12]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#Loading the Iris dataset



X, y = data["iq"].to_list(), data["Angle"].map(lambda x: {"0":0,
                                                "30":1,
                                                "60":2,
                                                "90":3,
                                                "120":4,
                                                "150":5,
                                                "180":6,
                                                }.get(str(x))).to_list()

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Creating an XGBoost classifier
model = xgb.XGBClassifier(enable_categorical=True)

#Training the model on the training data
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [13]:
#Making predictions on the test set
predictions = model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names={"0","30","60","90","120","150","180"}))

Accuracy: 0.9793918918918919

Classification Report:
              precision    recall  f1-score   support

          90       0.96      0.97      0.97       416
         150       0.98      0.96      0.97       421
           0       0.99      1.00      0.99       411
         120       0.99      0.99      0.99       417
         180       0.98      0.97      0.98       431
          30       0.97      1.00      0.99       434
          60       0.99      0.97      0.98       430

    accuracy                           0.98      2960
   macro avg       0.98      0.98      0.98      2960
weighted avg       0.98      0.98      0.98      2960



In [8]:
X_test[0]

[18,
 -186,
 190,
 23,
 -24,
 193,
 -190,
 -18,
 29,
 -189,
 189,
 38,
 -55,
 184,
 -182,
 -60,
 72,
 92,
 42,
 15,
 -42,
 -90,
 -47,
 -117,
 -34,
 -24,
 3,
 108,
 18,
 125,
 20,
 42,
 10,
 -95,
 18,
 -126,
 -15,
 -39,
 -25,
 94,
 -56,
 116,
 9,
 45,
 47,
 -82,
 65,
 -106,
 -1,
 -41,
 -72,
 65,
 -94,
 90,
 -18,
 44,
 76,
 -45,
 102,
 -63,
 26,
 -34,
 -93,
 17,
 -125,
 36,
 -35,
 27,
 93,
 -10,
 126,
 -7,
 35,
 -22,
 -99,
 -19,
 -117,
 -30,
 -41,
 14,
 88,
 43,
 111,
 55]

In [15]:
model.predict([X_test[0]])

array([6], dtype=int64)

In [5]:
model.predict([data.iloc[0]["iq"]])

array([0], dtype=int64)

In [17]:
import paho.mqtt.client as mqtt
import json
import numpy as np
import datetime


# Callback chamado quando o cliente recebe a resposta CONNACK do servidor.
def on_connect(client, userdata, flags, rc):
    print("Conectado com o código de resultado "+str(rc))
    client.subscribe("/gw/ac233fc09283/status")
    
# Callback chamado quando uma mensagem PUBLISH é recebida do servidor.
def on_message(client, userdata, msg):
    data = json.loads(msg.payload)
    prediction = model.predict([data[1]["aoa"]["iq"]])
    print({ "0":0,
            "1":30,
            "2":60,
            "3":90,
            "4":120,
            "5":150,
            "6":180}.get(str(prediction[0])))

client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect("broker.hivemq.com", 1883, 60)

now = datetime.datetime.now()
client.loop_forever()


150
120
150
150
120
90
120
150
150
120
120
150
120
120
150
120
150
150
30
120
120
120
150
90
150
150
150
120
150
120
120
120
0
0
120
150
150
120
150
150
120
30
150
90
150
120
120
120
120
120
150
150
120


KeyboardInterrupt: 

# Phase diffs  ( no reference )

In [5]:
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

#Loading the Iris dataset



X, y = data["iq"].map(lambda y: [ np.angle(complex(y[i-1], y[i])) for i in range(16,len(y),2)]).to_list(), data["Angle"].map(lambda x: {"0":0,
                                                "30":1,
                                                "60":2,
                                                "90":3,
                                                "120":4,
                                                "150":5,
                                                "180":6,
                                                }.get(str(x))).to_list()

#Splitting the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

#Creating an XGBoost classifier
model = xgb.XGBClassifier(enable_categorical=True)

#Training the model on the training data
model.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [6]:
#Making predictions on the test set
predictions = model.predict(X_test)

#Calculating accuracy
accuracy = accuracy_score(y_test, predictions)

print("Accuracy:", accuracy)
print("\nClassification Report:")
print(classification_report(y_test, predictions, target_names={"0","30","60","90","120","150","180"}))

Accuracy: 0.9658783783783784

Classification Report:
              precision    recall  f1-score   support

         120       0.96      0.96      0.96       437
          90       0.95      0.98      0.96       385
          30       0.99      0.99      0.99       419
         180       0.97      1.00      0.98       440
         150       0.98      0.92      0.95       419
           0       0.95      0.97      0.96       444
          60       0.97      0.95      0.96       416

    accuracy                           0.97      2960
   macro avg       0.97      0.97      0.97      2960
weighted avg       0.97      0.97      0.97      2960



In [8]:
data["iq"].iloc[0].__len__()

84